**Importação de Bibliotecas e Conjunto de Dados**

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar os dados
df_customers = pd.read_csv('/content/BankChurners.csv')

# Exibir as primeiras linhas para entender a estrutura
df_customers.head()


**Tratamento de Dados**

In [ ]:
# Verificar valores ausentes
missing_values = df_customers.isnull().sum()
print("Valores ausentes:\n", missing_values)

# Verificar se há dados duplicados
duplicate_rows = df_customers.duplicated().sum()
print("Registros duplicados: ", duplicate_rows)


**Análise Exploratória de Dados (EDA)**

Distribuição de Churn (Attrition_Flag)

In [ ]:
# Configurações de estilo
sns.set(style="whitegrid")

# Contagem da variável de churn (Attrition_Flag)
plt.figure(figsize=(8, 6))
sns.countplot(data=df_customers, x='Attrition_Flag', palette='Set2')
plt.title('Distribuição de Churn (Attrition_Flag)', fontsize=14)
plt.xlabel('Situação do Cliente', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.show()


**Distribuição por Gênero**

In [ ]:
# Distribuição por gênero
plt.figure(figsize=(8, 6))
sns.countplot(data=df_customers, x='Gender', palette='Set1', hue='Attrition_Flag')
plt.title('Distribuição de Gênero por Status de Churn', fontsize=14)
plt.xlabel('Gênero', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.show()


**Distribuição por Categoria de Renda**

In [ ]:
# Distribuição por categoria de renda
plt.figure(figsize=(10, 6))
sns.countplot(data=df_customers, x='Income_Category', palette='Set3', hue='Attrition_Flag')
plt.title('Distribuição de Categoria de Renda por Status de Churn', fontsize=14)
plt.xlabel('Categoria de Renda', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.xticks(rotation=45)
plt.show()


**Pré-processamento dos Dados**

Codificação das Variáveis Categóricas

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Criar uma cópia dos dados
df_processed = df_customers.copy()

# Codificar as variáveis categóricas
le = LabelEncoder()

categorical_cols = ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category', 'Attrition_Flag']
for col in categorical_cols:
    df_processed[col] = le.fit_transform(df_processed[col])

# Verificar se a codificação foi aplicada corretamente
df_processed.head()


**Divisão do Conjunto de Dados**

In [ ]:
from sklearn.model_selection import train_test_split

# Definir as variáveis X e Y
X = df_processed.drop(columns=['Attrition_Flag', 'CLIENTNUM'])  # Retirando a variável alvo e o identificador único
Y = df_processed['Attrition_Flag']

# Dividir em conjuntos de treino e teste (80% treino, 20% teste)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Conjunto de treino:", X_train.shape, Y_train.shape)
print("Conjunto de teste:", X_test.shape, Y_test.shape)


**Aplicação do SMOTE para Balanceamento**

In [ ]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE no conjunto de treino
smote = SMOTE(random_state=42)
X_train_smote, Y_train_smote = smote.fit_resample(X_train, Y_train)

# Verificar o balanceamento após SMOTE
print("Distribuição das classes após SMOTE:", Y_train_smote.value_counts())


**Treinamento e Avaliação de Modelos**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Criar e treinar o modelo de Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_smote, Y_train_smote)

# Prever no conjunto de teste
Y_pred = rf_model.predict(X_test)

# Avaliação do modelo
print("Matriz de Confusão:\n", confusion_matrix(Y_test, Y_pred))
print("\nRelatório de Classificação:\n", classification_report(Y_test, Y_pred))


**Análise da Curva ROC e AUC**

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Calcular as probabilidades de predição
Y_prob = rf_model.predict_proba(X_test)[:, 1]

# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(Y_test, Y_prob)
auc_score = roc_auc_score(Y_test, Y_prob)

# Plotar a curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Random Forest (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('Taxa de Falsos Positivos (FPR)', fontsize=12)
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)', fontsize=12)
plt.title('Curva ROC', fontsize=14)
plt.legend()
plt.show()
